In [1]:
from PIL import Image
import numpy as np
import cv2
import math
from matplotlib import pyplot as plt
import imutils

In [24]:
def rot_x(angle,ptx,pty):
    return math.cos(angle)*ptx + math.sin(angle)*pty

def rot_y(angle,ptx,pty):
    return -math.sin(angle)*ptx + math.cos(angle)*pty

def crop(image=None):
    data = np.asarray(image)

    non_empty_columns = np.where(data.max(axis=0)>0)[0]
    non_empty_rows = np.where(data.max(axis=1)>0)[0]

    cropBox = (min(non_empty_rows), max(non_empty_rows), min(non_empty_columns), max(non_empty_columns))

    data_crop = data[cropBox[0]:cropBox[1]+1, cropBox[2]:cropBox[3]+1]
    
    return Image.fromarray(data_crop)

In [70]:
def scale_and_rotate_image(im, sx, sy, deg_ccw, mode='RGBA', color=None):
    im_orig = im
    im = Image.new(mode, im_orig.size, color)
    im.paste(im_orig)

    w, h = im.size
    angle = math.radians(-deg_ccw)

    cos_theta = math.cos(angle)
    sin_theta = math.sin(angle)

    scaled_w, scaled_h = w * sx, h * sy

    new_w = int(math.ceil(math.fabs(cos_theta * scaled_w) + math.fabs(sin_theta * scaled_h)))
    new_h = int(math.ceil(math.fabs(sin_theta * scaled_w) + math.fabs(cos_theta * scaled_h)))

    cx = w / 2.
    cy = h / 2.
    tx = new_w / 2.
    ty = new_h / 2.

    a = cos_theta / sx
    b = sin_theta / sx
    c = cx - tx * a - ty * b
    d = -sin_theta / sy
    e = cos_theta / sy
    f = cy - tx * d - ty * e

    return im.transform(
        (new_w, new_h),
        Image.AFFINE,
        (a, b, c, d, e, f),
        resample=Image.NEAREST
    )

In [115]:
photo = Image.open("test/test_image.tif")
data = np.array(photo)
data[data == 0] = 2
data = (data / np.max(data)) * 255
photo = Image.fromarray(data)
photo = crop(photo)

<ipython-input-115-e51c96de7f72>:4: RuntimeWarning: overflow encountered in multiply
  data = (data / np.max(data)) * 255


In [119]:
n_zeros = np.Inf
for angle in np.arange(0, 20, 0.01):
    im = crop(scale_and_rotate_image(photo, 1, 1, angle, mode='I;16'))
    fit = np.sum(np.array(im) == 0)
    if fit < n_zeros:
        n_zeros = fit
        optimal_angle = angle
im_final = crop(scale_and_rotate_image(photo, 1, 1, optimal_angle, mode='I;16'))
Image.fromarray(np.array(im_final)*255).show()

In [118]:
Image.fromarray(np.array(im_final)*255).show()

In [92]:
cropped_image = crop(im)
cropped_image.show()
Image.fromarray(np.array(cropped_image)*255).show()


In [93]:
data = np.array(cropped_image)

In [120]:
-np.Inf

-inf

In [43]:
#data = np.asarray(crop(photo))
im = Image.fromarray(data)

(x, y) = im.size

angle = math.radians(12)

xextremes = [rot_x(angle,0,0),rot_x(angle,0,y-1),rot_x(angle,x-1,0),rot_x(angle,x-1,y-1)]
yextremes = [rot_y(angle,0,0),rot_y(angle,0,y-1),rot_y(angle,x-1,0),rot_y(angle,x-1,y-1)]
mnx = min(xextremes)
mxx = max(xextremes)
mny = min(yextremes)
mxy = max(yextremes)

T = np.matrix([[math.cos(angle),math.sin(angle),-mnx],[-math.sin(angle),math.cos(angle),-mny],[0,0,1]])
Tinv = np.linalg.inv(T);
Tinvtuple = (Tinv[0,0],Tinv[0,1], Tinv[0,2], Tinv[1,0],Tinv[1,1],Tinv[1,2])

im_transformed = im.transform(
    (int(round(mxx-mnx)),int(round((mxy-mny)))),Image.AFFINE,Tinvtuple,resample=Image.BILINEAR)

data = np.array(im_transformed)
data.shape

# im_transformed = crop(im_transformed)
# im_transformed.show()

(1238, 1228)

In [36]:
# rotate - opposite angle
T_rotate = np.array([
    [0, 1, 0],
    [, 0, 0],
    [0, 0, 1]])

T_translate = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]])

# Add up the transformations
T = T_translate @ T_rotate

T_inv = np.linalg.inv(T)
img_transformed = img.transform((n_col, n_row), Image.AFFINE, data=T_inv.flatten()[:6], resample=Image.NEAREST)
img_transformed.show()

LinAlgError: Singular matrix

In [11]:
cv2.drawContours(np.float32(image_new),[box],0,(0,0,255),10)
imgplot = plt.imshow(image_new)
plt.show()

NameError: name 'image_new' is not defined

In [6]:
photo = Image.open("test/test_image.tif")
data = np.array(photo)
no_data = data[0][0]
data[data == no_data] = np.NAN

(0, 0, 710, 695)

In [30]:
np.sum(~np.isnan(data[0][:]))

0

In [57]:
n_row, n_col = data.shape

### DO MIN ROW
data_count = 0
i = -1
while (data_count == 0):
    i = i + 1
    data_count = np.sum(~np.isnan(data[i][:]))

row_min = i

### DO MAX ROW
data_count = 0
i = n_row
while (data_count == 0):
    i = i - 1
    data_count = np.sum(~np.isnan(data[i][:]))

row_max = i

### DO MIN COL
data_count = 0
i = -1
while (data_count == 0):
    i = i + 1
    data_count = np.sum(~np.isnan(data[:][i]))

col_min = i

### DO MAX COL
data_count = 0
i = n_col
while (data_count == 0):
    i = i - 1
    data_count = np.sum(~np.isnan(data[:][i]))

col_max = i

In [124]:
n_row, n_col = data_new.shape
c1 = [0, np.min(np.where(data_new[0, :]>0))]
c2 = [np.max((np.where(data_new[:, 0]>0))),0]
c3 = [n_row-1, np.min(np.where(data_new[n_row-1, :]>0))]
c4 = [np.max(np.where(data_new[:, n_col-1]>0)), n_col-1]

box = np.array([c1, c2, c3, c4])
print(box)

[[  0 113]
 [596   0]
 [709 582]
 [120 694]]


In [110]:
np.where(data_new[:, 0]>0)

(array([596]),)

In [32]:
# loop over the rotation angles again, this time ensure the
# entire pill is still within the ROI after rotation
for angle in np.arange(0, 360, 15):
    rotated = imutils.rotate_bound(imageROI, angle)
    cv2.imshow("Rotated (Correct)", rotated)
    cv2.waitKey(0)

In [107]:
data_clip = data_new[0:200, 0:400]*255
Image.fromarray(data_clip).show()

<ipython-input-107-8ab3e1afe300>:1: RuntimeWarning: overflow encountered in multiply
  data_clip = data_new[0:200, 0:400]*255


In [34]:
data_2

array([[1, 2, 3, 4],
       [5, 6, 7, 8]])

In [35]:
data[0][:]

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na